In [1]:
import os
import logging
import pandas as pd
import dtale
import numpy as np
from sklearn.feature_selection import chi2, VarianceThreshold, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import eli5
from util import get_dialect
import homophone
import imp

logging.getLogger().setLevel(logging.INFO)
np.random.seed(77213)

In [2]:
prefix = r'D:\git\zhongguoyuyan\csv'
location = pd.read_csv(os.path.join(prefix, 'dialect', 'location.csv'), index_col=0)
char = pd.read_csv(os.path.join(prefix, 'words.csv'), index_col=0)
data = homophone.load_data(os.path.join(prefix, 'dialect'), location.index).reindex(char.index, axis=1, level=1)

INFO:root:loading 1287 data files ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K06mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K10mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K09mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K11mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K07mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K08mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F98mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F90mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F89mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F96mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F91mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F97mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F87mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\

In [3]:
train_loc, test_loc, train_data, test_data = train_test_split(location, data, test_size=0.2)
train_dialect = get_dialect(train_loc)
test_dialect = get_dialect(test_loc)

In [4]:
transformer = homophone.ColumnTransformer(transformers=[(
    col,
    homophone.make_pipeline(
        homophone.PhoneClustering(dtype=np.float32, n_clusters=200),
        homophone.Homophone(dtype=np.float32),
        homophone.VarianceThreshold(threshold=0.998 * (1 - 0.998)),
        homophone.HomophoneClustering(n_clusters=1000)
    ),
    train_data.columns.get_loc(col)
) for col in train_data.columns.levels[0]], sparse_threshold=1, n_jobs=3)
train_features = transformer.fit_transform(train_data)
feature_names = np.asarray(transformer.get_feature_names(char['item'].reindex(train_data.columns, level=1)))

In [5]:
idx = np.argsort(-VarianceThreshold(0).fit(train_features).variances_)[:100]
df = pd.DataFrame(train_features[:, idx].A, index=train_data.index, columns=feature_names[idx])
df.insert(0, 'dialect', train_dialect)
dtale.show(df, name='train_features_variance')

In [6]:
selector = SelectKBest(chi2, k=1000).fit(train_features[train_dialect != ''], train_dialect[train_dialect != ''])
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_features[np.ix_(train_dialect != '', idx)].A,
    index=train_data[train_dialect != ''].index,
    columns=feature_names[idx]
)
df.insert(0, 'dialect', train_dialect[train_dialect != ''])
dtale.show(df, name='train_features_chi2')

In [7]:
lr = LogisticRegression(penalty='l1', solver='saga', fit_intercept=False).fit(train_features[train_dialect != ''], train_dialect[train_dialect != ''])
test_features = transformer.transform(test_data)
print(classification_report(train_dialect[train_dialect != ''], lr.predict(train_features[train_dialect != ''])))
print(classification_report(test_dialect[test_dialect != ''], lr.predict(test_features[test_dialect != ''])))

              precision    recall  f1-score   support

        东北官话       0.87      1.00      0.93        27
        中原官话       0.97      0.95      0.96        62
        兰银官话       1.00      1.00      1.00        11
        冀鲁官话       0.96      0.96      0.96        23
        北京官话       1.00      1.00      1.00        11
         吴方言       0.98      1.00      0.99        47
        客家方言       0.97      0.94      0.96        34
          平话       0.88      0.78      0.82         9
         徽方言       1.00      1.00      1.00        11
         晋方言       1.00      0.97      0.99        35
        江淮官话       1.00      0.94      0.97        33
        湖南土话       0.00      0.00      0.00         1
         湘方言       0.90      0.93      0.91        28
         粤方言       0.95      0.97      0.96        38
        胶辽官话       0.91      0.71      0.80        14
        西南官话       0.99      1.00      0.99       147
         赣方言       0.97      1.00      0.98        57
         闽方言       0.96    

In [8]:
eli5.explain_weights(lr, top=10, feature_names=feature_names)

Explanation(estimator="LogisticRegression(fit_intercept=False, penalty='l1', solver='saga')", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='东北官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝岩盐难颜棉年人门匀云忙迎名赢零脓=吉橘竹菊足烛_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝岩盐难颜棉年人门匀云忙迎名赢零脓=壳格隔_吉橘竹菊足烛=河螺蛇华奴吴壶如余煤回怀时随肥楼浮流严嫌林兰连延言莲完顽还船圆原园民邻神银魂轮纯坟娘黄房防王能绳行明形荣营红熊雄浓龙容_吉橘竹菊足烛=茄婆爬茶斜图徐除锄台财排牌鞋赔皮池骑迟祠棋垂锤桃朝桥条头绸愁球潭蚕含咸钳甜寻沉琴弹钱缠前盘还全传权贫陈辰勤盆唇裙糖长床尝狂朋藤层棚平程城瓶停蓬铜虫穷_河螺蛇华奴吴壶如余煤回怀时随肥楼浮流严嫌林兰连延言莲完顽还船圆原园民邻神银魂轮纯坟娘黄房防王能绳行明形荣营红熊雄浓龙容=壳格隔_壳格隔=茄婆爬茶斜图徐除锄台财排牌鞋赔皮池骑迟祠棋垂锤桃朝桥条头绸愁球潭蚕含咸钳甜寻沉琴弹钱缠前盘还全传权贫陈辰勤盆唇裙糖长床尝狂朋藤层棚平程城瓶停蓬铜虫穷', weight=0.45275787593492073, std=None, value=None), FeatureWeight(feature='finals_歌个可鹅饿河课=破婆磨磨摸', weight=0.4184545421802192, std=None, value=None), FeatureWeight(feature='finals_拨泼末=车蛇射_拨泼末=折撤舌设_车蛇射=破婆磨磨摸_折撤舌设=破婆磨磨摸', weight=0.4113119314080689, std=None, value=None), FeatureWeight(feature='tone_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝岩盐难颜棉年人门匀云忙迎名赢零脓=夹_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝岩盐难颜棉年人门匀云忙迎名赢零脓=叔_锣鹅牙爷鱼来埋泥犁雷儿移眉梨姨围毛熬摇牛油南蓝岩盐难颜棉年人门匀云忙迎名赢零脓=摘_柴田=夹_柴田=叔_柴田=摘_河螺蛇华奴吴壶如余煤回怀时随肥楼浮流严嫌林兰连延言莲完顽还船圆原园民邻神银魂轮纯坟娘黄房防王能绳行明形荣营红熊雄浓龙容=夹_河螺蛇华奴吴壶如余煤回怀时随肥楼浮流严嫌林兰连延言莲完顽还船圆原园民邻神银魂轮纯坟娘黄房防王能绳行明形荣营红熊雄浓龙容=叔_河螺蛇华奴吴壶如余煤回怀时随肥楼浮流严嫌林兰连延言莲完顽还船圆原园民邻神银魂轮纯坟娘黄房防王能绳行明形荣营红熊雄浓龙容=摘_夹=茄婆爬茶斜图徐除锄台财排牌鞋赔皮池骑迟祠棋垂锤桃朝桥条头绸愁球潭蚕含咸钳甜寻沉琴弹钱缠前盘还全传权贫陈辰勤盆唇裙糖长床尝狂朋藤层棚平程城瓶停蓬铜虫穷_叔=茄婆爬茶斜图徐除锄台财排牌鞋赔皮池骑迟祠棋垂锤桃朝桥条头绸愁球潭蚕含咸钳甜寻沉琴弹钱缠前盘还全传权贫陈辰勤盆唇裙糖长床尝狂朋藤层棚平程城瓶停蓬铜虫穷_摘=茄婆爬茶斜图徐除锄台财排牌鞋赔皮池骑迟祠棋垂锤桃朝桥条头绸愁球潭蚕含咸钳甜寻沉琴弹钱缠前盘还全传权贫陈辰勤盆唇裙糖长床尝狂朋藤层棚平程城瓶停蓬铜虫穷', weight=0.3260443868571497, std=None, value=None), FeatureWeight(feature='tone_虾区圈村筐窗厅葱=猫_多拖歌靴沙车瓜花铺租箍乌猪初书输胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢关弯砖翻冤吞根恩亲新身筋墩蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方桩双江灯僧冰升生更坑争耕兵清贞声轻钉青星经兄东通公翁冬松风丰中终宫封蜂冲恭凶=猫', weight=0.297029749793174, std=None, value=None), FeatureWeight(feature='tone_可姐谱所府嫂表小感毯险品产典显隐党纺影整桶孔统=甲法角国_甲法角国=马野吕买尾母有懒满暖软两冷领_甲法角国=鸟猛永_甲法角国=躲锁果火把假哑写赌土古苦虎鼠举许取主改海摆解矮洗拐紫纸比死子使喜几嘴水鬼宝讨早草好饱找抖走狗口酒手九胆敢减点铲板剪浅扁短碗选卷反紧本滚笋准粉抢想厂响绑讲等肯打省梗饼井顶挺懂肿_甲法角国=瓦五耳老藕远网_甲法角国=女武米李脑染眼引', weight=0.29670359666354357, std=None, value=None)], neg=[FeatureWeight(feature='initial_假嫁锯交减夹监甲奸江讲=街解_街解=戒间角', weight=-0.5181436604181109, std=None, value=None), FeatureWeight(feature='initial_鸡寄饥记几季叫九剑金急建肩见结紧吉筋劲姜脚镜经击=街解_街解=举句卷决均橘军菊', weight=-0.4949270699991348, std=None, value=None), FeatureWeight(feature='initial_及杰健极局=街解_轿舅旧件近=街解', weight=-0.38316126199186096, std=None, value=None), FeatureWeight(feature='tone_吉橘竹菊足烛=笔骨北得色谷福宿曲_吉橘竹菊足烛=吸七虱失一出黑织屋_笔骨北得色谷福宿曲=刻侧测_笔骨北得色谷福宿曲=吸七虱失一出黑织屋_鸽接贴歇切结泼脱缺约剥劈踢锡=甲法角国_鸽接贴歇切结泼脱缺约剥劈踢锡=铁节雪血脚百尺_鸽接贴歇切结泼脱缺约剥劈踢锡=发各恶客策_甲法角国=铁节雪血脚百尺_甲法角国=插鸭擦八扎杀瞎刷桌_铁节雪血脚百尺=发各恶客策_铁节雪血脚百尺=插鸭擦八扎杀瞎刷桌_刻侧测=吸七虱失一出黑织屋_发各恶客策=发各恶客策_发各恶客策=插鸭擦八扎杀瞎刷桌', weight=-0.31364930540866015, std=None, value=None)], pos_remaining=69, neg_remaining=35), proba=None, score=None, weighted_spans=None, heatmap=None), TargetExplanation(target='中原官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_叶辣热月落药麦=虾区圈村筐窗厅葱_叶辣热月落药麦=多拖歌靴沙车瓜花铺租箍乌猪初书输胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢关弯砖翻冤吞根恩亲新身筋墩蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方桩双江灯僧冰升生更坑争耕兵清贞声轻钉青星经兄东通公翁冬松风丰中终宫封蜂冲恭凶_虾区圈村筐窗厅葱=日木目_虾区圈村筐窗厅葱=立入密力鹿绿_虾区圈村筐窗厅葱=业灭列孽末越弱_多拖歌靴沙车瓜花铺租箍乌猪初书输胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢关弯砖翻冤吞根恩亲新身筋墩蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方桩双江灯僧冰升生更坑争耕兵清贞声轻钉青星经兄东通公翁冬松风丰中终宫封蜂冲恭凶=日木目_多拖歌靴沙车瓜花铺租箍乌猪初书输胎该开街低梯西鸡溪杯灰歪碑知资师饥丝希衣吹规亏追龟飞刀高包抓抄交敲焦烧腰箫偷钩修抽州休优丢贪三甘衫尖签添心参针深金音单肝安山间班奸鲜天先肩牵烟搬端酸官宽欢关弯砖翻冤吞根恩亲新身筋墩蹲孙婚温春均分军熏帮汤仓糠张装疮霜章伤姜秧光慌方桩双江灯僧冰升生更坑争耕兵清贞声轻钉青星经兄

Executing shutdown due to inactivity...


2022-04-11 17:13:14,092 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-04-11 17:13:45,138 - INFO     - Executing shutdown...
